In [17]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm
from sqlalchemy import create_engine
import psycopg2

In [18]:
# Get names of newspapers from the newseum's website
newseum = "http://www.newseum.org/todaysfrontpages/?tfp_display=list&tfp_region=USA&tfp_sort_by=state"
page = urllib.request.urlopen(newseum)
root = BeautifulSoup(page, 'html.parser')
res = root.find_all('a', {'class': 'tfp-list-link'})
names = [x.text.split('\n')[1] for x in res]
links = [x['href'] for x in res]
loc_name = [x.find('h6').text for x in res]
city_name = [x.split(',')[0] for x in loc_name]
state_name = [re.findall(', ([A-Za-z. ]*)', x)[0][:-4] for x in loc_name]
local_news = pd.DataFrame(list(zip(names, links, city_name, state_name)), columns=['name', 'link', 'city', 'state'])

In [31]:
state_map = """Alaska	AK
Ala.	AL
Ark.	AR
Ariz.	AZ
Calif.	CA
Colo.	CO
Conn.	CT
D.C.	DC
Del.	DE
Fla.	FL
Ga.	GA
Hawaii	HI
Iowa	IA
Idaho	ID
Ill.	IL
Ind.	IN
Kan.	KS
Ky.	KY
La.	LA
Mass.	MA
Md.	MD
Maine	ME
Mich.	MI
Minn.	MN
Mo.	MO
Miss.	MS
Mont.	MT
N.C.	MC
N.D.	ND
Neb.	NE
N.H.	NH
N.J.	NJ
N.M.	NM
Nev.	NV
N.Y.	NY
Ohio	OH
Okla.	OK
Ore.	OR
Pa.	PA
R.I.	RI
S.C.	SC
S.D.	SD
Tenn.	TN
Texas	TX
Utah	UT
Va.	VA
Vt.	VT
Wash.	WA
Wis.	WI
W. Va.	WV
Wyo.	WY"""

state_map = dict(zip([x.split('\t')[0] for x in state_map.split('\n')], [x.split('\t')[1] for x in state_map.split('\n')]))
local_news['state_code'] = local_news['state'].map(lambda x: state_map[x])

In [ ]:
def getOuterLink(newseum_link):
    try:
        newseum_link = "http://www.newseum.org/" + newseum_link
        x = urllib.request.urlopen(newseum_link)
        x = BeautifulSoup(x, 'html.parser')
        return x.find_all('span', {'class': 'fa fa-external-link'})[0].parent['href']
    except:
        return None
    
local_news['outer_links'] = local_news['link'].map(getOuterLink)

In [42]:
local_news.to_csv('./local_news.csv', encoding='utf-8')

In [6]:
#You will need to store your db credentials in a file in a dictionary named "creds"
exec(open('./db.cred').read())

In [14]:
engine = create_engine('postgresql://{user}:{password}@{host}/{dbname}'.format(**creds)
                       , echo=False)

In [50]:
local_news.columns = ['name', 'link', 'city', 'state', 'url', 'state_code']

In [64]:
local_news['id'] = local_news.index

In [72]:
local_news[['name', 'city', 'state_code', 'url' ]].to_sql(name='local_news', con=engine, if_exists='replace', 
                                                          index=True, index_label='id')

In [18]:
conn = psycopg2.connect("dbname='{dbname}' user='{user}' host='{host}' password='{password}'".format(**creds))

In [23]:
# Make these directories
FRONT_PAGES = './front_pages'
OP_EDS = './op_eds'

In [65]:
def getHeadline(link, file_name):
    r = requests.get(link, allow_redirects=True)
    open('%s/%s' % (FRONT_PAGES, file_name), 'wb').write(r.content)

def getOpinion(link, file_name):
    r = requests.get(link + '/opinion', allow_redirects=True)
    open('%s/%s' % (OP_EDS, file_name), 'wb').write(r.content)


In [66]:
logfile = open('pull_pages.log', 'w')
errorfile = open('pull_pages.error', 'w')

In [67]:
for x in tqdm(sample.iterrows()):
    link = x[1]['outer_links']
    name = x[1]['name']
    try:
        getHeadline(link, name.lower().replace(' ', '_') + '.html')
    except:
        errorfile.write('Missed headline for %s\n' % name)
    try:
        getOpinion(link, name.lower().replace(' ', '_') + '.html')
    except:
        errorfile.write('Missed opeds for %s\n' % name)
    errorfile.flush()
    logfile.write("%s\n" % name)
    logfile.flush()

5it [00:01,  2.82it/s]
